# Pymatgen core functionality

In [17]:
from pythreejs import *
from IPython.display import display
from pymatgen.vis.structure_vtk import EL_COLORS

el_color_mapping = EL_COLORS["VESTA"]


def struct_to_rend(structure):
    big_structure = structure.copy()
    big_structure.make_supercell([2, 2, 2])
    balls = []
    for site in big_structure:
        color = "#%02x%02x%02x" % tuple(el_color_mapping[site.specie.symbol])
        balls += [Mesh(geometry=SphereGeometry(radius=site.specie.average_ionic_radius),
                       material=LambertMaterial(color=color),
                       position = site.coords.tolist())]
    scene = Scene(children=balls)
    camera = PerspectiveCamera(position = [0, 5, 5], up = [0, 0, 1],
                               children = [DirectionalLight(color='white', 
                                                            position = [3, 5, 1], 
                                                            intensity = 0.5)])
    renderer = Renderer(camera = camera, scene=scene, controls=[OrbitControls(controlling=camera)])
    display(renderer)

In [18]:
struct_to_rend(structure)

## Structures, sites, and lattices

In [7]:
from pymatgen import *
import pymatgen
mpr = MPRester()
%matplotlib inline

nacl_structs = mpr.get_structures("mp-680944")
len(nacl_structs)
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
sga = SpacegroupAnalyzer(nacl_structs[0])
conv = sga.get_conventional_standard_structure()
conv.to(filename="Nb2O5.cif")

In [8]:
# Making lattices
lattice = Lattice([[2.8, 0, 0], [0, 2.8, 0], [0, 0, 2.8]])
lattice.cubic(2.8)
lattice.hexagonal(a = 2.8, c = 3.6)
lattice.rhombohedral(a = 2.8, alpha = 60)

# Manipulating lattices
print lattice.a

# Making structures
bcc_fe = Structure(lattice, ["Fe", "Fe"], [[0, 0, 0], [0.5, 0.5, 0.5]])
site0 = bcc_fe[0]
site0.coords
site0.species_string
site0.x

nacl= Structure(lattice, ["Na+", "Cl-"], [[0, 0, 0], [0.5, 0.5, 0.5]])

bcc_fe = Structure.from_spacegroup("Im-3m", Lattice.cubic(2.8), ["Fe"], [[0, 0, 0]])
print(bcc_fe)
big_structure = Structure.from_file("Nb2O5.cif")
big_structure.formula

# Making disordered structures
specie = {"Cu0+": 0.5, "Au0+":0.5}
cu_au = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.677), [specie], [[0, 0, 0]])
print(cu_au)


# Manipulating structures and assigning properties to sites
big_structure[0] = "V"
big_structure.formula
big_structure[0] = "Nb"

bcc_fe.append("C", [0.25, 0.25, 0.25])
bcc_fe.pop(-1)
bcc_fe.make_supercell([2, 2, 2])

sd = []
names = []
for n in range(big_structure.num_sites):
    if big_structure[n].species_string == "Nb":
        sd.append([False, False, False])
        names.append("Jerry")
    else:
        sd.append([True, True, True])
        names.append("Billy Mays")
big_structure.add_site_property("selective_dynamics", sd)
big_structure.to(filename="POSCAR")
big_structure.add_site_property("names", names)
site0 = big_structure[0]

2.8
Full Formula (Fe2)
Reduced Formula: Fe
abc   :   2.800000   2.800000   2.800000
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0.5  0.5  0.5
Full Formula (Cu2 Au2)
Reduced Formula: CuAu
abc   :   3.677000   3.677000   3.677000
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP                        a    b    c
---  ----------------------  ---  ---  ---
  0  Cu0+:0.500, Au0+:0.500  0    0    0
  1  Cu0+:0.500, Au0+:0.500  0    0.5  0.5
  2  Cu0+:0.500, Au0+:0.500  0.5  0    0.5
  3  Cu0+:0.500, Au0+:0.500  0.5  0.5  0


### Exercise: find your favorite oxide using the materials project rester.  Replace each oxygen atom with fluorine.

In [9]:
# Potential solution:
structure = mpr.get_structures("BaNiO3")[0]
for n in range(structure.num_sites):
    if structure[n].species_string == 'O':
        structure[n] = 'F'
    
#print(structure.formula)
#print(structure)

## Transformations

In [15]:
# Using transformations
from pymatgen.transformations.standard_transformations import SubstitutionTransformation, DeformStructureTransformation, \
OrderDisorderedStructureTransformation, RotationTransformation
structure = mpr.get_structures("BaNiO3")[0]
st = SubstitutionTransformation({"O":"F"})
new_structure = st.apply_transformation(structure)
old_structure = st.inverse.apply_transformation(new_structure)
old_structure == structure

# Order disorder
odst = OrderDisorderedStructureTransformation()
ss = odst.apply_transformation(cu_au)
len(ss)

4

## SymmetryAnalyzer and StructureMatcher

### Exercise: Perform a transformation on a structure and use the structure matcher on the result.

In [16]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.structure_matcher import StructureMatcher

structure = mpr.get_structures("BaNiO3")[0]
sga = SpacegroupAnalyzer(structure)

sga.get_conventional_standard_structure()
rt = RotationTransformation([1, 0, 0], 120)
st = SubstitutionTransformation({"O":"F"})

new_structure = rt.apply_transformation(structure)
sm = StructureMatcher()
print(sm.fit(structure, new_structure))

new_structure = st.apply_transformation(structure)
print(sm.fit(structure, new_structure))
print(sm.fit_anonymous(structure, new_structure))

True
None
True


## XRD, Bandstructure, and Density of States

### Exercise: Plot the XRD spectra, Bandstructure, and electronic DOS of Au

In [ ]:
from pymatgen.analysis.diffraction.xrd import XRDCalculator
from pymatgen.electronic_structure.plotter import BSPlotter, DosPlotter
from pymatgen.electronic_structure.core import OrbitalType

data = mpr.get_data('TiO2')
structures = mpr.get_structures("TiO2")
xrd_calc = XRDCalculator()
#xrd_calc.show_xrd_plot(structures[0])

bs = mpr.get_bandstructure_by_material_id(data[0]["material_id"])
print(bs.get_band_gap())
plotter=BSPlotter(bs)
#plotter.get_plot().show()
#plotter.plot_brillouin()

dos = mpr.get_dos_by_material_id(data[0]["material_id"])
dp = DosPlotter()
dos_ti = dos.get_element_spd_dos("Ti")
dos_o = dos.get_element_spd_dos("O")
dp.add_dos("O p-states", dos_o[OrbitalType.p])
dp.add_dos("Ti d-states", dos_ti[OrbitalType.d])
dp.get_plot().show()

## Phase diagrams

### Exercise: Plot phase diagrams for the Ba-Ni-O chemical system

In [ ]:
from pymatgen.phasediagram.maker import PhaseDiagram
from pymatgen.phasediagram.plotter import PDPlotter

entries = mpr.get_entries_in_chemsys(["Ba", "Ni", "O"])
pd = PhaseDiagram(entries)
plotter = PDPlotter(pd, show_unstable=True) # show_unstable=true
plotter.show()

## Tensors

### Fit a "noisy" tensor to a particular crystal structure

## Surfaces

### Generate all of the low-index facets for BCC Fe

In [ ]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.surface import generate_all_slabs
lattice = mg.Lattice.cubic(2.85)
structure = mg.Structure(lattice, ["Fe", "Fe"], 
                         [[0, 0, 0], [0.5, 0.5, 0.5]])

slabs = generate_all_slabs(structure, 1, 4, 10)
first_slab = slabs[0]
first_slab.miller_index
for slab in slabs:
    print slab.miller_index

## Pourbaix Diagrams?

## Etc.

In [ ]:
data = mpr.get_data("mp-149")
et = data[0]["elasticity"]["elastic_tensor"]
structure = Structure.from_str(data[0]["cif"], fmt="cif")